# Classification Algorithm Analysis
Notebook to analyze data generated by `classification_algo` in `crawler.py`.

## Setup

In [2]:

import os
import pandas as pd
import json
import statistics
from pathlib import Path
import yaml
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib as mpl
from filelock import FileLock
from crawler import CrawlResults
from utils.utils import get_directories, get_domain, split
from utils.image_shingle import ImageShingle
import time
import numpy as np
import math

mpl.rcParams.update(mpl.rcParamsDefault)

CRAWL_NAME = 'KJ2GW'
NUM_WORKERS = 25

DATA_PATH = Path("/usr/project/xtmp/mml66/cookie-classify/") / CRAWL_NAME
ANALYSIS_PATH = Path("analysis") / CRAWL_NAME
FIGURE_PATH = Path("analysis") / CRAWL_NAME / "figs"
FIGURE_PATH.mkdir(parents=True, exist_ok=True)
(ANALYSIS_PATH / "slurm/differences").mkdir(parents=True, exist_ok=True)

# Config
with open(DATA_PATH / "config.yaml", "r") as stream:
    config = yaml.safe_load(stream)

# Site list
site_list = []
with open(config["SITE_LIST_PATH"]) as file:
    for line in file:
        site_list.append(line.strip())

# Site queue
queue_lock = FileLock(config["QUEUE_PATH"] + '.lock', timeout=10)
with queue_lock:
    with open(config["QUEUE_PATH"], 'r') as file:
        site_queue = json.load(file)

# Site results
results_lock = FileLock(config["RESULTS_PATH"] + '.lock', timeout=10)
with results_lock:
    with open(config["RESULTS_PATH"]) as file:
        site_results: dict[str, CrawlResults] = json.load(file)

"""
Check crawl completion.
"""
print(f"Crawled {len(site_results)}/{len(site_list)} sites.")

"""
Reduce the number of sites to analyze.
A successful site must have:
1. a successful domain -> url resolution
3. was not terminated via SIGKILL
2. no unexpected crawl exceptions
"""
successful_sites = []
unsuccessful_sites = []
keys = set()
for domain, result in site_results.items():
    keys.update(result.keys())
    if result.get("url") and not result.get("SIGKILL") and not result.get("unexpected_exception"):
        successful_sites.append(domain)
    else:
        unsuccessful_sites.append(domain)
print(f"{len(successful_sites)} successful sites.")

##############################################################################

## Unsuccessful sites stats

In [ ]:
unsuccessful_sites_stats = {
    "landing_page_down": 0,
    "unexpected_exception": 0,
    "SIGKILL": 0,
    "SIGTERM": 0,
}
for domain in unsuccessful_sites:
    result: CrawlResults = site_results[domain]
    for key in unsuccessful_sites_stats.keys():
        if result.get(key):
            unsuccessful_sites_stats[key] += 1
print(unsuccessful_sites_stats)
print(f"Total unsuccessful sites: {len(unsuccessful_sites)}")

## Successful sites stats

In [ ]:
successful_sites_stats = {
    "landing_page_down": 0,
    "SIGTERM": 0,
}
for domain in successful_sites:
    result: CrawlResults = site_results[domain]
    for key in successful_sites_stats.keys():
        if result.get(key):
            successful_sites_stats[key] += 1
print(successful_sites_stats)
print(f"Total successful sites: {len(successful_sites)}")

## Plot Helper Functions

In [ ]:
def capitalize_keys(original_dict):
    """
    Capitalizes all keys of the given dictionary.

    Args:
        original_dict (dict): The original dictionary with keys to be capitalized.

    Returns:
        dict: A new dictionary with all keys capitalized.
    """
    return {key.capitalize(): value for key, value in original_dict.items()}

def generate_pie_chart(freq_dict, title=None):
    """
    Generates a pie chart from a frequency dictionary.

    Args:
        freq_dict (dict): A dictionary where keys are categories and values are frequencies.
        title (str): The title of the pie chart. If None, no title is displayed.

    Returns:
        None: Displays the pie chart.
    """
    freq_dict = capitalize_keys(freq_dict)
    
    # Sort the keys alphabetically
    sorted_keys = sorted(freq_dict.keys())

    # Generate a list of distinct colors
    # Note: Increase the number of colors if you have more categories
    colors = ['gold', 'lightcoral', 'lightskyblue', 'lightgreen', 'lavender', 'orange', 'cyan', 'pink', 'yellowgreen', 'plum']

    # Create a color map by assigning each key a color
    color_map = {key: colors[i % len(colors)] for i, key in enumerate(sorted_keys)}

    # Extract and sort the labels (keys) and sizes (values) from the frequency dictionary based on sorted labels
    labels = sorted_keys
    sizes = [freq_dict[key] for key in sorted_keys]
    
    # Get the list of colors based on the order of sorted labels
    pie_colors = [color_map[label] for label in labels]

    # Generate the pie chart
    plt.figure(figsize=(4, 4))  # Set the figure size
    plt.pie(sizes, labels=labels, colors=pie_colors, autopct='%1.1f%%', startangle=40)
    plt.axis('equal')  # Equal aspect ratio ensures the pie chart is circular.
    
    if title:
        plt.title(title)

In [ ]:
def get_cdf_df(values: list):
    """
    Returns a dataframe which can be plotted as a CDF graph (see plot_cdf).

    args:
        values: list of values to plot
    """
    s = pd.Series(values, name = 'value')
    df = pd.DataFrame(s)

    # Frequency
    stats_df = df \
    .groupby('value') \
    ['value'] \
    .agg('count') \
    .pipe(pd.DataFrame) \
    .rename(columns = {'value': 'frequency'})

    # PDF
    stats_df['pdf'] = stats_df['frequency'] / sum(stats_df['frequency'])

    # CDF
    stats_df['cdf'] = stats_df['pdf'].cumsum()
    stats_df = stats_df.reset_index()

    return stats_df

def plot_cdf(data: dict[str, pd.DataFrame], axis: list):
    """
    Plots multiple CDF graphs on the same axis.

    Args:
        values_list: list of lists, each containing values to plot
        labels: list of labels for the graphs
        axis: axis limits
    """    
    plt.figure(figsize=(6, 4))
    for label, values in data.items():
        df = get_cdf_df(values)
        plt.plot(df['value'], df['cdf'], label=label, alpha=0.8)

    plt.axis(axis)

    plt.ylabel('CDF')
    
    plt.grid(visible=True)
    plt.legend()


def plot_pdf(values: list, label: str, axis: list):
    """
    Plots a PDF graph.

    Args:
        values: list of values to plot
        label: label for the graph
        axis: axis limits
    """    
    mpl.rcParams.update(mpl.rcParamsDefault)

    df = get_cdf_df(values)

    plt.plot(df['value'], df['pdf'], label=label)

    plt.axis(axis)

    plt.xlabel(label)
    plt.ylabel('PDF')

    plt.grid(visible = True)

    plt.show()

### Time

In [ ]:
times = []
for site in site_list:
    times.append(site_results[site]["total_time"])
print(f"Median time to crawl a site: {statistics.median(times) / 60} minutes.")
print(f"Total time to crawl all sites: {sum(times) / (60 * 60 * 24)} days.")

Per worker

In [ ]:
total_time = {}
for domain, result in site_results.items():
    task_id = result["SLURM_ARRAY_TASK_ID"]
    total_time[task_id] = total_time.get(task_id, 0) + (result["total_time"] / (60 * 60))  # convert to hours

# Convert the dictionary into a DataFrame
df = pd.DataFrame(list(total_time.items()), columns=['SLURM_ARRAY_TASK_ID', 'Total_Crawl_Time_Hours'])

# Sort the DataFrame by 'Total_Crawl_Time' in ascending order
df_sorted = df.sort_values(by='Total_Crawl_Time_Hours', ascending=True)

# Display the sorted DataFrame
print(df_sorted.to_string(index=False))

### Clickstream length
Statisistics for both clickstream generation and traversal.

Elements clicked by type

In [ ]:
from itertools import chain

click_types = {}
for site in successful_sites:
    for _, click_type in list(chain.from_iterable(site_results[site]['clickstream'])):
        click_types[click_type] = click_types.get(click_type, 0) + 1

click_types

Failed elements clicked by type.
This means we were able to generate the clickstream, but unable to traverse it again.

In [ ]:
click_failures = {}

for site in successful_sites:
    for type, failures in site_results[site]['traversal_failures'].items():
        click_failures[type] = click_failures.get(type, 0) + failures

click_failures

In [ ]:
generate_pie_chart(click_types, title="All Clickable Elements")
plt.savefig(FIGURE_PATH / f'all_clickable_elements.png', dpi=600, facecolor='white')
generate_pie_chart(click_failures, title="Failed Clickable Elements")
plt.savefig(FIGURE_PATH / f'failed_clickable_elements.png', dpi=600, facecolor='white')

Average clickstream length for each group.

- For baseline, this is the average length of the generated clickstream.
- For control and experimental, this is the average length of the successful traversal.

In [ ]:
def count_actions(path: Path, name: str) -> int:
    """
    Counts the number of actions in a clickstream by counting the number of images in the directory.
    """    
    count = 0
    while (path / f"{name}-{count+1}.png").is_file():
        count += 1
    return count

In [ ]:
# # Uncomment to rebuild
# def get_average_clickstream_length_per_site() -> pd.DataFrame:
#     """
#     Returns a dataframe with the average number of actions for each site.
    
#     Should not be used to compute the average clickstream length for the entire crawl
#     since we cannot take the average of averages.
#     """    
#     rows_list = []

#     for i, domain in enumerate(successful_sites):
#         print(f"Processing site {i+1}/{len(successful_sites)}: {domain}")
#         clickstreams = get_directories(site_results[domain]["data_path"])

#         baseline_actions = []
#         control_actions = []
#         experimental_actions = []
#         for clickstream in clickstreams:
#             if (clickstream / f"baseline-0.png").is_file() and \
#                 (clickstream / f"control-0.png").is_file() and \
#                     (clickstream / f"experimental-0.png").is_file():
#                 baseline_actions.append(count_actions(clickstream, "baseline"))
#                 control_actions.append(count_actions(clickstream, "control"))
#                 experimental_actions.append(count_actions(clickstream, "experimental"))
        
#         if len(baseline_actions) != 0 and len(control_actions) != 0 and len(experimental_actions) != 0:
#             rows_list.append({
#                 "domain": domain,
#                 "baseline": statistics.mean(baseline_actions),
#                 "control": statistics.mean(control_actions),
#                 "experimental": statistics.mean(experimental_actions)
#             })

#     return pd.DataFrame(rows_list)

# clickstream_lengths = get_average_clickstream_length_per_site()
# clickstream_lengths['baseline/control'] = clickstream_lengths['baseline'] / clickstream_lengths['control']

# # Sort by baseline/control ratio and remove 0/0
# clickstream_length_by_domain = clickstream_lengths.sort_values(by='baseline/control', ascending=False)
# clickstream_length_by_domain.dropna(inplace=True)
# clickstream_length_by_domain.to_csv(ANALYSIS_PATH / "clickstream_length_by_domain.csv", index=False)
# clickstream_length_by_domain

In [ ]:
clickstream_length_by_domain = pd.read_csv(ANALYSIS_PATH / 'clickstream_length_by_domain.csv')
clickstream_length_by_domain

In [ ]:
# Uncomment to rebuild

# def get_all_clickstream_lengths() -> tuple[list, list, list]:
#     """
#     Returns a tuple of lists containing the lengths of all clickstreams for each type.

#     Returns: baseline_actions, controls_actions, experimental_actions
#     """
#     baseline_actions = []
#     control_actions = []
#     experimental_actions = []
#     for i, domain in enumerate(successful_sites):
#         print(f"Processing site {i+1}/{len(successful_sites)}: {domain}")
        
#         clickstreams = get_directories(site_results[domain]["data_path"])

#         for clickstream in clickstreams:
#             # Ensure that we've navigated to the landing page for all three groups
#             if (clickstream / f"baseline-0.png").is_file() and \
#                     (clickstream / f"control-0.png").is_file() and \
#                         (clickstream / f"experimental-0.png").is_file():
#                 baseline_actions.append(count_actions(clickstream, "baseline"))
#                 control_actions.append(count_actions(clickstream, "control"))
#                 experimental_actions.append(count_actions(clickstream, "experimental"))
    
#     return baseline_actions, control_actions, experimental_actions

# baseline_actions, control_actions, experimental_actions = get_all_clickstream_lengths()

# with open(ANALYSIS_PATH / 'baseline_clickstream_lengths.json', 'w') as f:
#     json.dump(baseline_actions, f)
# with open(ANALYSIS_PATH / 'control_clickstream_lengths.json', 'w') as f:
#     json.dump(control_actions, f)
# with open(ANALYSIS_PATH / 'experimental_clickstream_lengths.json', 'w') as f:
#     json.dump(experimental_actions, f)

In [ ]:
with open(ANALYSIS_PATH / 'baseline_clickstream_lengths.json', 'r') as f:
    baseline_actions = json.load(f)
with open(ANALYSIS_PATH / 'control_clickstream_lengths.json', 'r') as f:
    control_actions = json.load(f)
with open(ANALYSIS_PATH / 'experimental_clickstream_lengths.json', 'r') as f:
    experimental_actions = json.load(f)

In [ ]:
print(f"Average baseline length: {statistics.mean(baseline_actions)}")
print(f"Average control length: {statistics.mean(control_actions)}")
print(f"Average experimental length: {statistics.mean(experimental_actions)}")

def plot_normalized_frequencies(lists_of_numbers, labels):
    """
    Plots the normalized frequencies of numbers as percentages from multiple lists with custom labels,
    with reduced whitespace between each set of bars for a more compact visualization.
    
    Parameters:
    lists_of_numbers (list of list of int): A list containing multiple lists of numbers.
    labels (list of str): A list containing labels for each list of numbers.
    """
    # Initialize an empty list to store normalized frequencies for each list
    all_normalized_frequencies = []

    # Function to calculate frequency
    def calculate_frequency(numbers):
        frequency = {}
        for number in numbers:
            if number in frequency:
                frequency[number] += 1
            else:
                frequency[number] = 1
        return frequency

    # Function to normalize frequency
    def normalize_frequency(frequency):
        total_count = sum(frequency.values())
        return {number: (count / total_count) for number, count in frequency.items()}

    # Calculating and normalizing frequencies for each list
    for numbers_list in lists_of_numbers:
        frequency = calculate_frequency(numbers_list)
        normalized_frequency = normalize_frequency(frequency)
        all_normalized_frequencies.append(normalized_frequency)

    # Combining all numbers from all lists and sorting them
    all_numbers = sorted(set().union(*[frequency.keys() for frequency in all_normalized_frequencies]))

    # Preparing the plot
    plt.figure(figsize=(6, 4))
    # Adjusting the width for reduced whitespace
    width = 0.35
    indices = [x * (1 + width) for x in range(len(all_numbers))]

    for i, normalized_frequency in enumerate(all_normalized_frequencies):
        data = [normalized_frequency.get(number, 0) for number in all_numbers]
        plt.bar([index + (i * width) - (width * len(lists_of_numbers) / 2) + width / 2 for index in indices], data, width=width, label=labels[i] if i < len(labels) else f'List {i+1}')

    plt.xlabel('Clickstream Length')
    plt.ylabel('Proportion')
    plt.ylim(0, 1)
    plt.title('Clickstream Length across Crawl Groups')
    plt.yticks(np.arange(0, 1+0.1, 0.1))
    plt.xticks(indices, all_numbers)
    plt.legend().set_title("Crawl Group")
    plt.grid(axis='y')

    # Showing the plot
    plt.savefig(FIGURE_PATH / 'clickstream_length.png', bbox_inches='tight', dpi=600, facecolor='white')
plot_normalized_frequencies([baseline_actions, control_actions, experimental_actions], ["Baseline", "Control", "Experimental"])

## Analysis

In [ ]:
raw_differences = {}
for i in range(NUM_WORKERS):
    with open(ANALYSIS_PATH / f'slurm/differences/{i}.json', 'r') as file:
        to_merge = json.load(file)
    raw_differences.update(to_merge)
print(f"Successful domains extracted: {len(raw_differences)}/{len(successful_sites)}")

def unravel(data: dict, diff_type: str, add_labels: bool = False) -> tuple[list, dict]:
    """
    Unravel the screenshot differences, filtered by clicksteam depth.

    Args:
        screenshots: The screenshots dictionary. See screenshot_comparison.py for the schema.
        difference_type: The type of difference to unravel.

    Returns:
        Tuple[list, dict]: A tuple containing a list of all the differences and a dictionary containing the differences by clickstream depth.
    """
    # Init
    # Schema: depth -> [domain, difference] | [difference]
    unraveled_raw: dict[int, pd.DataFrame | list] = {}  # Unravel each difference by depth
    unraveled_domains: dict[int, pd.DataFrame | list] = {}  #  # Average differences by domain before unraveling
    for i in list(range(config["CLICKSTREAM_LENGTH"]+1)) + ["ALL"]:
        unraveled_raw[i] = []
        unraveled_domains[i] = []

    # Unravel
    for domain, clickstreams in data.items():
        domain_diffs = {}  # collect all diffs for a given domain by depth
        for clickstream, actions in clickstreams.items():
            for action, diff_dict in actions.items():
                action = int(action)
                
                if diff_type in diff_dict:
                    # Append to domain_diffs
                    domain_diffs[action] = domain_diffs.get(action, []) + [diff_dict[diff_type]]
                    domain_diffs["ALL"] = domain_diffs.get("ALL", []) + [diff_dict[diff_type]]

        # Unravel all diffs into results
        for depth, diffs in domain_diffs.items():
            if add_labels:
                unraveled_raw[depth].extend([{"domain": domain, diff_type: diff} for diff in diffs])
                unraveled_domains[depth].append({"domain": domain, diff_type: statistics.mean(diffs)})
            else:
                unraveled_raw[depth].extend(diffs)
                unraveled_domains[depth].append(statistics.mean(diffs))
                
    # Convert dict to dataframe
    if add_labels:
        for depth, diffs in unraveled_domains.items():
            unraveled_domains[depth] = pd.DataFrame(diffs)
        for depth, diffs in unraveled_raw.items():
            unraveled_raw[depth] = pd.DataFrame(diffs)
        
    return {"unraveled_raw": unraveled_raw, "unraveled_domains": unraveled_domains}

diff_types = ["bce_diff", "shingle_did", "img_did", "innerText_did", "links_did"]

unraveled_raw = {}
unraveled_domains = {}
for diff_type in diff_types:
    unraveled_raw[diff_type] = unravel(raw_differences, diff_type)["unraveled_raw"]
    unraveled_domains[diff_type] = unravel(raw_differences, diff_type)["unraveled_domains"]
    
diffs_df = pd.DataFrame(list(successful_sites), columns=['domain'])
for diff_type in diff_types:
    to_merge = unravel(raw_differences, diff_type, add_labels=True)["unraveled_domains"]["ALL"]
    diffs_df = diffs_df.merge(to_merge, on='domain', how='left')

diffs_df.to_csv(ANALYSIS_PATH / "differences.csv", index=False)
diffs_df

## Screenshot Difference CDF

In [ ]:
names = {
    "bce_diff": "Screenshot",
}

for diff_type in names:
    axis = [0, 0.5, 0, 1]
    plt.figure(figsize=(6, 4))
    for label, values in unraveled_domains[diff_type].items():
        df = get_cdf_df(values)
        plt.plot(df['value'], df['cdf'], label=label, alpha=0.8)
    plt.axis(axis)
    plt.ylabel('Cumulative Probability')
    plt.yticks(np.arange(0, 1+0.1, 0.1))
    plt.grid(visible=True)
    plt.legend()
    plt.title(names[diff_type])
    plt.xlabel('BCE Screenshot Difference')
    plt.legend().set_title("Depth")
    plt.savefig(FIGURE_PATH / f'{diff_type}.png', bbox_inches='tight', dpi=600, facecolor='white')

In [ ]:
names = {
    "shingle_did": "Image Shingles",
    "innerText_did": "Inner Text",
    "img_did": "Images",
    "links_did": "Links",
}

# Setup the figure and axes for a 2x2 grid
fig, axs = plt.subplots(2, 2, figsize=(12, 8))  # Adjust the size as needed
axs = axs.flatten()  # Flatten to ease the iteration

for i, (diff_type, name) in enumerate(names.items()):
    axis = [-0.2, 0.2, 0, 1]
    axs[i].set_title(name)
    axs[i].set_xlabel('Difference in Difference')
    axs[i].set_ylabel('Cumulative Probability')
    axs[i].set_yticks(np.arange(0, 1+0.1, 0.1))
    axs[i].grid(visible=True)
    axs[i].axis(axis)
    
    for label, values in unraveled_domains[diff_type].items():
        df = get_cdf_df(values)
        axs[i].plot(df['value'], df['cdf'], label=label, alpha=0.8)
        
    axs[i].legend(loc='upper left').set_title("Depth")

# Adjust layout to prevent overlap
plt.tight_layout()

# Save the combined figure
plt.savefig(FIGURE_PATH / 'combined_jaccard_did.png', bbox_inches='tight', dpi=600, facecolor='white')

In [ ]:
from scipy.stats import norm

names = {
    "shingle_did": "Image Shingles",
    "innerText_did": "Inner Text",
    "img_did": "Images",
    "links_did": "Links",
}

# Setup the figure and axes for a 2x2 grid
fig, axs = plt.subplots(2, 2, figsize=(12, 8))  # Adjust the size as needed
axs = axs.flatten()  # Flatten to ease the iteration

for i, (diff_type, label) in enumerate(names.items()):
    data = unraveled_domains[diff_type]["ALL"]

    # Fit a normal distribution to the data:
    mu, std = norm.fit(data)

    # Plot the histogram on subplot
    axs[i].hist(data, bins=50, density=True, alpha=0.6, label='Histogram')

    # Plot the PDF on subplot
    xmin, xmax = axs[i].get_xlim()
    x = np.linspace(xmin, xmax, 100)
    p = norm.pdf(x, mu, std)
    axs[i].plot(x, p, 'k', linewidth=2, label='Gaussian Fit: μ=%.2f, σ=%.2f' % (mu, std))
    
    # Setting title, labels, and legend
    axs[i].set_title(label)
    axs[i].set_xlabel('Difference in Difference')
    axs[i].set_ylabel('Probability Density')
    axs[i].legend(loc='upper left')
    axs[i].axis([-1, 1, 0, 35])

# Adjust layout to prevent overlap
plt.tight_layout()

plt.savefig(FIGURE_PATH / 'combined_fit_gaussian.png', bbox_inches='tight', dpi=600, facecolor='white')